In [1]:
# Thank you, Daisy! I love the NYPD Collision Walkthrough - A great way to refresh on Python / Pandas

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import glob
print(os.listdir("data/"))

import tqdm
tqdm.tqdm.pandas()

import operator
import functools

import matplotlib.pyplot as plt
import seaborn as sns



['JC-201701-citibike-tripdata.csv', 'JC-201902-citibike-tripdata.csv', 'JC-201801-citibike-tripdata.csv', 'JC-201709-citibike-tripdata.csv', 'JC-201809-citibike-tripdata.csv', 'JC-201803-citibike-tripdata.csv', 'JC-201703-citibike-tripdata.csv', 'JC-201708 citibike-tripdata.csv', 'JC-201608-citibike-tripdata.csv', 'JC-201606-citibike-tripdata.csv', 'JC-201705-citibike-tripdata.csv', 'JC-201610-citibike-tripdata.csv', 'JC-201805-citibike-tripdata.csv', 'JC-201906-citibike-tripdata.csv', 'JC-201811-citibike-tripdata.csv', 'JC-201807-citibike-tripdata.csv', 'JC-201904-citibike-tripdata.csv', 'JC-201707-citibike-tripdata.csv', 'JC-201711-citibike-tripdata.csv', 'JC-201612-citibike-tripdata.csv', 'JC-201808-citibike-tripdata.csv', 'JC-201903-citibike-tripdata.csv', 'JC-201609-citibike-tripdata.csv', 'JC-201901-citibike-tripdata.csv', 'JC-201802-citibike-tripdata.csv', 'JC-201702-citibike-tripdata.csv', 'JC-201712-citibike-tripdata.csv', 'JC-201611-citibike-tripdata.csv', 'JC-201607-citibike

In [2]:
path = 'data' # use your path

# read in all the csvs

all_files = glob.glob(path + "/*.csv")

# empty list to store all csv dataframes for concatenation

li = []

for csv in all_files:
    df = pd.read_csv(csv, index_col=None, header=0)
    
    # The columns in some CSVs are title case, make all lowercase to merge
    df.columns = df.columns.str.lower()
    
    # The columns in some CSVs have spaces in titles, for those that do, remove the space to merge
    df_rename = df.rename(columns = {'bike id':'bikeid', 'start time' : 'starttime', 'stop time': 'stoptime', 'trip duration':'tripduration', 'user type':'usertype'})
    
    # For the formatted dataframes, send it to the empty list
    li.append(df_rename)

# Concatenate all the dataframes into one    
bike_dataframe = pd.concat(li, axis=0, ignore_index = True, sort=True)

# View the first five rows
bike_dataframe.head()

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype
0,24575,1983.0,3185,40.717733,-74.043845,City Hall,1,3276,40.714584,-74.042817,Marin Light Rail,2017-01-01 00:21:32,2017-01-01 00:24:01,148,Subscriber
1,24723,1978.0,3198,40.748716,-74.040443,Heights Elevator,1,3183,40.716247,-74.033459,Exchange Place,2017-01-01 00:24:35,2017-01-01 00:45:58,1283,Subscriber
2,24620,1989.0,3211,40.721525,-74.046305,Newark Ave,1,3183,40.716247,-74.033459,Exchange Place,2017-01-01 00:38:19,2017-01-01 00:44:31,372,Subscriber
3,24668,1961.0,3271,40.692640,-74.088012,Danforth Light Rail,1,3194,40.725340,-74.067622,McGinley Square,2017-01-01 00:38:37,2017-01-01 01:03:50,1513,Subscriber
4,26167,1993.0,3203,40.727596,-74.044247,Hamilton Park,1,3183,40.716247,-74.033459,Exchange Place,2017-01-01 01:47:52,2017-01-01 01:58:31,639,Subscriber


In [3]:
#familiarizing oneself with the data, mostly trying to see which columns have unexpected uniques
display(bike_dataframe.shape)
display(bike_dataframe.dtypes)

# display(bike_dataframe['end station id'].nunique())
# display(bike_dataframe['end station name'].nunique())
# display(bike_dataframe['end station latitude'].nunique())
# display(bike_dataframe['end station longitude'].nunique())
# display(bike_dataframe['start station id'].nunique())
# display(bike_dataframe['start station name'].nunique())
# display(bike_dataframe['start station latitude'].nunique())
# display(bike_dataframe['start station longitude'].nunique())

# # display(bike_dataframe['end station id'].value_counts().sort_index(ascending=True) )
# # display(bike_dataframe['start station id'].value_counts().sort_index(ascending=True) )

# display(bike_dataframe.loc[bike_dataframe['end station id'] == 127])
# testr = bike_dataframe.loc[(bike_dataframe['end station latitude'] >= 40.73) & (bike_dataframe['end station latitude'] <= 40.7338)]

# testr['end station name'].value_counts()


(1001802, 15)

bikeid                       int64
birth year                 float64
end station id               int64
end station latitude       float64
end station longitude      float64
end station name            object
gender                       int64
start station id             int64
start station latitude     float64
start station longitude    float64
start station name          object
starttime                   object
stoptime                    object
tripduration                 int64
usertype                    object
dtype: object

In [4]:


starttime_rename = bike_dataframe["starttime"].str.split(" ", n = 1, expand = True)
bike_dataframe['startdate'] = starttime_rename[0]
bike_dataframe['starttime'] = starttime_rename[1]

stoptime_rename = bike_dataframe["stoptime"].str.split(" ", n = 1, expand = True)
bike_dataframe['stopdate'] = stoptime_rename[0]
bike_dataframe['stoptime'] = stoptime_rename[1]

bike_dataframe.head()

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype,startdate,stopdate
0,24575,1983.0,3185,40.717733,-74.043845,City Hall,1,3276,40.714584,-74.042817,Marin Light Rail,00:21:32,00:24:01,148,Subscriber,2017-01-01,2017-01-01
1,24723,1978.0,3198,40.748716,-74.040443,Heights Elevator,1,3183,40.716247,-74.033459,Exchange Place,00:24:35,00:45:58,1283,Subscriber,2017-01-01,2017-01-01
2,24620,1989.0,3211,40.721525,-74.046305,Newark Ave,1,3183,40.716247,-74.033459,Exchange Place,00:38:19,00:44:31,372,Subscriber,2017-01-01,2017-01-01
3,24668,1961.0,3271,40.692640,-74.088012,Danforth Light Rail,1,3194,40.725340,-74.067622,McGinley Square,00:38:37,01:03:50,1513,Subscriber,2017-01-01,2017-01-01
4,26167,1993.0,3203,40.727596,-74.044247,Hamilton Park,1,3183,40.716247,-74.033459,Exchange Place,01:47:52,01:58:31,639,Subscriber,2017-01-01,2017-01-01


In [10]:
# Gender (Zero=unknown; 1=male; 2=female)
# User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)

bike_dataframe_rename = bike_dataframe.replace({'gender': {0: 'Unknown', 1: 'Male', 2:'Female'}})
bike_dataframe_rename.head()

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype,startdate,stopdate
0,24575,1983.0,3185,40.717733,-74.043845,City Hall,Male,3276,40.714584,-74.042817,Marin Light Rail,00:21:32,00:24:01,148,Subscriber,2017-01-01,2017-01-01
1,24723,1978.0,3198,40.748716,-74.040443,Heights Elevator,Male,3183,40.716247,-74.033459,Exchange Place,00:24:35,00:45:58,1283,Subscriber,2017-01-01,2017-01-01
2,24620,1989.0,3211,40.721525,-74.046305,Newark Ave,Male,3183,40.716247,-74.033459,Exchange Place,00:38:19,00:44:31,372,Subscriber,2017-01-01,2017-01-01
3,24668,1961.0,3271,40.692640,-74.088012,Danforth Light Rail,Male,3194,40.725340,-74.067622,McGinley Square,00:38:37,01:03:50,1513,Subscriber,2017-01-01,2017-01-01
4,26167,1993.0,3203,40.727596,-74.044247,Hamilton Park,Male,3183,40.716247,-74.033459,Exchange Place,01:47:52,01:58:31,639,Subscriber,2017-01-01,2017-01-01


In [11]:
bike_dataframe_rename.to_csv('bikedata_june2016toJune2019.csv')